Brice Le Pape

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Support Vector Elastic Net</div>

0. [Préambule](#preambule)
1. [Introduction](#intro)
2. [LASSO](#lasso)
3. [Elastic Net](#elasticnet)
4. [SVM](#svm)
5. [Réduction d'Elastic Net à SVM](#reduction)
6. [Implémentation de SVEN](#sven)
7. [Optimisation](#more)

[Sources](#sources)

## <a id="preambule"></a> 0. Préambule

J'ai écrit ce notebook en 2018 lors de ma dernière année à l'[ISAE-SUPAERO](https://www.isae-supaero.fr/), en filière [Sciences des Données et de la Décision](https://supaerodatascience.github.io/), dans le cadre du cours de Machine Learning d'[Emmanuel Rachelson](https://people.isae-supaero.fr/emmanuel-rachelson). Il était demandé aux élèves d'écrire un cours sur un sujet donné de Machine Learning, et de le mettre à disposition des autres élèves au format notebook. Le sujet qui m'a été donné était : "Beyond SVM and back : from LASSO to Elastic Net". Ce notebook est très largement inspiré de la publication [A Reduction of the Elastic Net to Support Vector Machines with an Application to GPU Computing](https://arxiv.org/pdf/1409.1976) par Quan Zhou, Wenlin Chen, Shiji Song, Jacob R. Gardner, Kilian Q. Weinberger et Yixin Chen.


## <a id="intro"></a> 1. Introduction

L'Elastic Net et le LASSO (qui est un cas particulier de l'Elastic Net) sont deux des algorithmes les plus utilisés ces dernières années notamment pour la selection de variables. L'augmentation de la taille des datasets a du coup fait augmenter les besoins d'implémentations rapides des techniques populaires de machine learning. Dans de nombreux domaines, comme en Imagerie par Résonnance Magnétique Fonctionnelle ou dans l'étude du génome, on peut facilement se retrouver avec plusieurs millions de variables.

En même temps, il est de plus en plus facile d'avoir accès à des processeurs GPU ou des CPU multi-coeurs, ce qui amène naturellement à vouloir améliorer les algorithmes par la parallélisation. Et bien sûr certains algorithmes sont plus faciles à paralléliser que d'autres. Par exemple, on sait que les réseaux de neurones profonds s'adaptent très bien sur des GPU, de même pour les Machines à Vecteurs de Support (SVMs).

Dans ce notebook, nous reprendrons les bases en redéfinissant le LASSO et l'Elastic Net, puis nous ferons de même avec les SVMs, et ensuite nous verrons comment nous pouvons réduire un problème Elastic Net (et LASSO à moindres mesures) à un problème SVM. Finalement, nous sortirons de ce notebook l'algorithme SVEN (Support Vector Elastic Net), qui permet de résoudre un problème Elastic Net tout en profitant du côté parallélisable des SVMs.


### A. Notations

À travers ce notebook nous noterons : 
- les vecteurs en **gras** (e.g. $\mathbf{x}$);
- les matrices en **GRAS MAJUSCULES** (e.g. $\mathbf{X}$);
- les scalaires en "normal" (e.g. $a$ or $B$);
- $x_i$ pour parler de la $i^{\text{ème}}$ valeur du vecteur $\mathbf{x}$;
- $\mathbf{x}^{(i)}$ pour parler de la $i^{\text{ème}}$ colonne de la matrice $\mathbf{X}$;
- $\mathbf{x}_i$ pour parler de la $i^{\text{ème}}$ ligne transposée de la matrice $\mathbf{X}$;
- $n$ ou $m$ le nombre d'exemples;
- $p$ or $q$ le nombre de variables (dimension).



### B. Datasets 

Nous allons travailler sur le dataset "*iris*" pour tester nos algorithmes. Ce dataset a une caractéristique $n > p$ avec n = 150 exemples et p = 4 variables. Sans l'illustrer concrètement, nous expliquerons ce qu'il se passe dans le cas intéressant où $p >> n$ (nous n'avons pas d'exemple ici car nous n'avons pas trouvé de dataset intéressant suffisamment petit pour être exploitable dans le temps imparti et avec la puissance de nos ordinateurs). 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
# iris dataset retrieval
from sklearn.datasets import load_iris
iris = load_iris()
scaler = StandardScaler()
data = scaler.fit_transform(iris['data'])
target = np.array([1. if i == 1 else -1. for i in iris['target']]).reshape((-1, 1))

print(data.shape)

In [ ]:
# visualization
colors = ['r' if l[0] > 0 else 'b' for l in target]

fig = plt.figure(figsize=(7, 7))
plt.scatter(data[:, 0], data[:, 3], color=colors)
plt.show()

Préparons des sets d'entrainement et de test sur notre jeu de données.

In [ ]:
from sklearn.model_selection import train_test_split

# We add a 'constant' column in the data matrix
X = np.append(np.ones((len(data), 1)), data, axis=1)

# We separate our dataset into a train set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=42)

<div class="alert alert-info"> 

##### Note 1

À cause du temps qui nous est imparti et de la puissance de calcul que nous disposons, nous avons fait le choix de travailler sur un "petit" dataset "visuel" qui n'a pas forcèment une application concrète. Cependant voici les liens vers des datasets plus intéressants, libre à vous de les utiliser pour la suite du notebook :

- $n >> p$ : [YMSD](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd) ($n = 515345, p=90$)

- $p >> n$ : [ARCENE](https://archive.ics.uci.edu/ml/datasets/Arcene) ($n=900, p=10000$) 
</div>

<div class="alert alert-info">

##### Note 2

Même si nous allons être amenés à travailler avec des algorithmes de régression, nous avons choisi de travailler avec une dataset qui est un problème de classification. Cela n'est pas grave, à ce moment là, il suffira de considérer nos labels comme des targets et de travailler avec une métrique adaptée.
  
</div>

## <a id="lasso"></a> 2. LASSO

### A. Définition (*Rappel*)

LASSO (pour Least Absolute Shrinkage and Selection Operator) est une méthode de régression qui pratique une régularisation et une sélection de variables pour améliorer les prédictions et l'interprétabilité des modèles statistiques qu'elle produit. Cette méthode a été d'abord introduite sur des travaux de géophysique en 1986, puis plus tard redécouverte et popularisée par Robert Tibshirani en 1996. 

Dans un problème de régression, nous avons un jeu de données $(\mathbf{X}, \mathbf{y})$, avec $\mathbf{X}$ la matrice standardisée de taille $n \times (p + 1) $ ($n$ exemples and $p$ variables $+ 1$ constante), et $\mathbf{y}$ le vecteur cible.

La méthode LASSO apprend un modèle linéaire pour prévoir $y_i$ à partir de $\mathbf{x}_i$ en minimisant une fonction de coût carrée avec une contrainte L1-normée.

$$\min_{\mathbf{\beta} \in \mathbb{R}^{p+1}} ||\mathbf{X \beta} − \mathbf{y}||_2^2 \text{ tel que } |\mathbf{\beta}|_1 \leq t $$

où $\mathbf{\beta} = [\beta_0, \beta_1, . . . , \beta_p]^T \in \mathbb{R}^{p+1}$ est le vecteur des coefficients, $t > 0$ le "budget" sur la L1-norme.  Pour rappel, la L1-norme $|.|_1$ est défini par $|\mathbf{\beta}|_1 = \sum_{j = 0}^p |\beta_j|$, et la L2-norme $||.||_2$ est défini par $||\mathbf{\beta}||_2 = \sqrt{\sum_{j = 0}^p \beta_j^2}$.

<div class="alert alert-warning"> 
    
### Exercice :

**Implémentez une fonction `LASSO` qui réalise la méthode LASSO et qui renvoie le vecteur $\beta$**

*Conseil : Utilisez la fonction `scipy.optimize.fmin_slsqp`. Elle résoud des problèmes d'optimisation non-linéaire sous contraintes par méthode des moindres carrées séquentielle. Elle prend en entrée :*
- *une fonction `func` du vecteur variable `x`,*
- *une valeur initiale `x0` pour `x`,*
- *une fonction `f_eqcons` qui retourne un vecteur de valeurs correspondant aux contraintes d'égalité `f_eqcons(x)==0`,*
- *a function `f_ieqcons` qui retourne un vecteur de valeurs correspondant aux contraintes d'inégalité `f_ieqcons(x)>=0`.*

</div>

In [ ]:
from scipy.optimize import fmin_slsqp

In [ ]:
# %load solutions/code1.py
### Ecrivez votre code ici
# Si vous etes bloques, decommentez la premiere ligne et executez ce bloc.

#def LASSO(X, y, t):

Testons notre algorithme

In [ ]:
from time import time
from sklearn.metrics import mean_squared_error

In [ ]:
# We process the lasso algorithm
t0 = time()
lasso_beta = LASSO(X_train, y_train, t=1.0)
print("> Time spent : {}s".format(time()-t0))

y_train_pred = np.dot(X_train, lasso_beta)
y_test_pred = np.dot(X_test, lasso_beta)
print("> Training set MSE = {}".format(mean_squared_error(y_train_pred, y_train)))
print("> Testing set MSE = {}\n".format(mean_squared_error(y_test_pred, y_test)))

<div class="alert alert-info">
    
##### Note

On pourrait trouver un $t$ optimal à notre avec une méthode de validation croisée.

</div>

In [ ]:
# visualization
train_colors = ['r' if l[0] > 0 else 'b' for l in y_train_pred]

test_colors = ['r' if l[0] > 0 else 'b' for l in y_test_pred]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.scatter(X_train[:, 1], X_train[:, 4], color=train_colors)
ax1.set_title("Training set")

ax2.scatter(X_test[:, 1], X_test[:, 4], color=test_colors)
ax2.set_title("Training set")

plt.show()

Ajouter un paragraphe sur la selection de features et les avantages et inconvénients du LASSO

### B. Avantages et limites

Les principaux avantages du LASSO sont :

- Grande dimension : le LASSO fonctionne dans les cas où le nombre d'exemples est inférieur au nombre de variables ($p > n$).

- Sélection de variables : le LASSO permet de sélectionner un sous-ensemble restreint de variables (dépendant du paramètre t). Cette sélection restreinte permet souvent de mieux interpréter un modèle.

Par contre, certaines limites du LASSO ont été démontrées :

- Les fortes corrélations : si des variables sont fortement corrélées entre elles et qu'elles sont importantes pour la prédiction, le LASSO en privilégiera une au détriment des autres. 

- La très grande dimension : lorsque $p >> n$ le LASSO ne pourra pas forcément retrouver l'ensemble de ces variables d'intérêts.

## <a id="elasticnet"></a> 3. Elastic Net

Pour dépasser les limitations rencontrées par le LASSO, on introduit l'Elastic Net, qui est une méthode de régression régularisée qui combine les pénalités sur la L1-norme et la L2-norme des méthodes LASSO et RIDGE respectivement.

Les limitations de LASSO sont dépassées par l'ajout d'un terme quadratique à la régularisation (qui revient à la méthode RIDGE si il est utilisé seul). Cela s'explique par le fait que le terme quadratique de pénalité rend la fonction de coût strictement convexe, qui présente donc un unique minimum. 

Comme pour LASSO, dans un problème de régression nous avons un jeu de données $(\mathbf{X}, \mathbf{y})$, avec $\mathbf{X}$ la matrice standardisée de taille $n \times (p + 1) $ ($n$ exemples and $p$ variables $+ 1$ constante), et $\mathbf{y}$ le vecteur cible.

La méthode Elastic Net apprend un modèle linéaire pour prévoir $y_i$ à partir de $\mathbf{x}_i$ en minimisant une fonction de coût carrée avec une contrainte L1-normée.

$$\min_{\mathbf{\beta} \in \mathbb{R}^{p+1}} ||\mathbf{X \beta} − \mathbf{y}||_2^2 + \lambda_2 ||\mathbf{\beta}||_2^2 \text{ tel que } |\mathbf{\beta}|_1 \leq t $$

où $\mathbf{\beta} = [\beta_0, \beta_1, . . . , \beta_p]^T \in \mathbb{R}^{p+1}$ est le vecteur des coefficients, $\lambda_2 \geq 0 $ est la constante de L2-régularisation, et $t > 0$ est le "budget" pour la L1-norme. 

<div class="alert alert-warning"> 
    
### Exercice :

**Implémentez une fonction `ElasticNet` qui réalise la méthode Elastic Net et qui renvoie le vecteur $\beta$**

*Conseil : Utilisez la même fonction `scipy.optimize.fmin_slsqp`.*

</div>

In [ ]:
# %load solutions/code2.py
### Ecrivez votre code ici
# Si vous etes bloques, decommentez la premiere ligne et executez ce bloc.

#def ElasticNet(X, y, t, l2):

Testons notre algorithme

In [ ]:
# We process the Elastic Net algorithm
t0 = time()
en_beta = ElasticNet(X_train, y_train, t=2.0, l2=0.5)
print("> Time spent : {}s".format(time()-t0))

y_train_pred = np.dot(X_train, en_beta)
y_test_pred = np.dot(X_test, en_beta)
print("> Training set MSE = {}".format(mean_squared_error(y_train_pred, y_train)))
print("> Testing set MSE = {}\n".format(mean_squared_error(y_test_pred, y_test)))

In [ ]:
# visualization
train_colors = ['r' if l[0] > 0 else 'b' for l in y_train_pred]

test_colors = ['r' if l[0] > 0 else 'b' for l in y_test_pred]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.scatter(X_train[:, 1], X_train[:, 3], color=train_colors)
ax1.set_title("Training set")

ax2.scatter(X_test[:, 1], X_test[:, 3], color=test_colors)
ax2.set_title("Training set")

plt.show()

## <a id="svm"></a> 4. SVM 

### A. Définition (*Rappel*)

Les Machines à Vecteurs de Support (SVMs) sont des modèles d'apprentissage supervisé adaptés pour la classification et la régression. Un modèle SVM peut être vu comme une représentation des exemples comme des points d'un espace, cartogagraphié de sorte à ce que les exemples venant de différentes classes soient séparés par un hyperplan avec une marge entre les deux classes la plus large possible. 

Pour plus de détails sur les SVMs, référez-vous au cours ["SVMs, bias-variance, and kernels" course](https://github.com/erachelson/MLclass/tree/master/3%20-%20SVMs%2C%20bias-variance%2C%20and%20kernels) d'Emmanuel Rachelson. Nous allons ici rapidement réintroduire le concept et le décliner sous ses formes primale et duale.

Dans un problème de classification, nous avons un jeu de données $(\mathbf{X}, \mathbf{y})$, avec $\mathbf{X} \in \mathbb{R}^{m \times q}$ la matrice standardisé de taile $m \times q$ ($m$ exemples et $q$ variables), et $\mathbf{y} \in \{-1, +1\}^m$ le vecteur label.

Un problème SVM consiste à apprendre à partir de ces données un hyperplan de séparation, paramétrisé par un vecteur de coefficients $\mathbf{w} \in \mathbb{R}^q$ et une constante $b \in \mathbb{R}$, avec une fonction de coût "square hinge":

$$\min_{\mathbf{w} \in \mathbb{R}^q, b \in \mathbb{R}} \frac{1}{m} \sum_{i=1}^m \max(0, 1-y_i(\mathbf{w}.\mathbf{x}_i - b)) + \lambda ||\mathbf{w}||_2^2$$

où $\lambda > 0$ est la constante de L2-régularisation. 

### B. Problème primal

Le problème d'optimisation précédent peut être réécrit comme un problème d'optimisation sous contraintes avec une fonction objectif différentiable de la façon suivante : 

Pour chaque $i\in \{1,\ldots ,m\}$ nous introduisons la variable $\zeta _{i}=\max(0,1-y_{i}(\mathbf{w} . \mathbf{x}_{i}-b))$. Notons que $\zeta _{i}$ est le plus petit nombre positif satisfaisant $y_{i}(\mathbf{w} . \mathbf{x}_{i}-b) \geq 1-\zeta _{i}$. Aussi nous définissons $C = \frac{1}{2 \lambda m}$.

Alors nous pouvons redéfinir le problème d'optimisation comme suit : 

$$\min_{\mathbf{w} \in \mathbb{R}^q, b \in \mathbb{R}} C \sum _{i=1}^{m}\zeta _{i}+ \frac{1}{2} ||\mathbf{w}||_2^2$$
$$\text{ sujet à  } y_{i}(\mathbf{w} . \mathbf{x}_{i}-b) \geq 1-\zeta _{i}\,{\text{ et }}\,\zeta _{i}\geq 0,\,{\text{pour tout }}i.$$ 

Ce problème est appelé *problème primal*.

<div class="alert alert-warning"> 
    
### Exercice :

**Implémentez une fonction `SVMPrimal` qui résoud le problème SVM primal et qui renvoie le vecteur $w$**

*Conseil : Utilisez la même fonction `scipy.optimize.fmin_slsqp`.*

</div>

In [ ]:
# %load solutions/code3.py
### Ecrivez votre code ici
# Si vous etes bloques, decommentez la premiere ligne et executez ce bloc.

#def SVMPrimal(X, y, C):

Testons maintenant notre algorithme.

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# We process the SVMPrimal algorithm
t0 = time()
primal_w = SVMPrimal(X_train, y_train, C=0.8)
print("> Time spent : {}s".format(time()-t0))

y_train_pred = np.array([1 if l[0] > 0 else -1 for l in np.dot(X_train, primal_w)])
y_test_pred = np.array([1 if l[0] > 0 else -1 for l in np.dot(X_test, primal_w)])
print("> Training set Accuracy = {}".format(accuracy_score(y_train_pred, y_train)))
print("> Testing set Accuracy = {}\n".format(accuracy_score(y_test_pred, y_test)))

In [ ]:
# visualization
train_colors = ['r' if l > 0 else 'b' for l in y_train_pred]

test_colors = ['r' if l > 0 else 'b' for l in y_test_pred]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.scatter(X_train[:, 1], X_train[:, 3], color=train_colors)
ax1.set_title("Training set")

ax2.scatter(X_test[:, 1], X_test[:, 3], color=test_colors)
ax2.set_title("Training set")

plt.show()

### C. Dual problem

En resolvant le [problème dual](https://en.wikipedia.org/wiki/Duality_(optimization)) du problème précédent, on obtient la simplification suivante :

$$ \min_{\alpha_i \geq 0} ||\mathbf{Z\alpha}||^2_2 + \frac{1}{2C} \sum_{i=1}^m \alpha_i^2 - 2 \sum_{i=1}^m \alpha_i$$

avec $\mathbf{\alpha} = (\alpha_1, \ldots, \alpha_m)$ les variables duales et $\mathbf{Z} = (y_1\mathbf{x}_1, \ldots, y_m\mathbf{x}_m)$ la matrice de taille $q \times m$, dont la $i^{ème}$ colonne $\mathbf{z}^{(i)}$ consiste en l'exemple $\mathbf{x}_i$ multiplié par son label correspondant $y_i$, i.e. $\mathbf{z}^{(i)} = y_i \mathbf{x}_i$. 

Ce problème est appelé *problème dual*.

Les problèmes primal et dual sont équivalent et les solutions sont liées par $\mathbf{w} = \sum_{i=1}^m y_i \alpha_i \mathbf{x}_i$.

<div class="alert alert-warning"> 
    
### Exercice :


**Implémentez une fonction `SVMDual` qui résoud le problème SVM dual et qui renvoie le vecteur $\alpha$**

*Conseil : Utilisez la même fonction `scipy.optimize.fmin_slsqp`.*

</div>

In [ ]:
# %load solutions/code4.py
### Ecrivez votre code ici
# Si vous etes bloques, decommentez la premiere ligne et executez ce bloc.

#def SVMDual(X, y, C):

Testons l'algorithme

In [ ]:
# We process the SVMDual algorithm
t0 = time()
dual_alpha = SVMDual(X_train, y_train, C=0.8)
dual_w = np.dot(X_train.T, y_train * dual_alpha)
print("> Time spent : {}s".format(time()-t0))

y_train_pred = np.array([1 if l[0] > 0 else -1 for l in np.dot(X_train, dual_w)])
y_test_pred = np.array([1 if l[0] > 0 else -1 for l in np.dot(X_test, dual_w)])
print("> Training set Accuracy = {}".format(accuracy_score(y_train_pred, y_train)))
print("> Testing set Accuracy = {}\n".format(accuracy_score(y_test_pred, y_test)))

In [ ]:
# visualization
train_colors = ['r' if l > 0 else 'b' for l in y_train_pred]

test_colors = ['r' if l > 0 else 'b' for l in y_test_pred]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.scatter(X_train[:, 1], X_train[:, 3], color=train_colors)
ax1.set_title("Training set")

ax2.scatter(X_test[:, 1], X_test[:, 3], color=test_colors)
ax2.set_title("Training set")

plt.show()

## <a id="reduction"></a> 5. Réduction d'Elastic Net à SVM


### A. Reformulation d'Elastic Net

Commençons de la formulation d'Elastic Net telle qu'on l'avait posé dans la section "[3. Elastic Net](#elasticnet)":

$$\min_{\mathbf{\beta}} ||\mathbf{X \beta} − \mathbf{y}||_2^2 + \lambda_2 ||\mathbf{\beta}||_2^2 \text{ tel que } |\mathbf{\beta}|_1 \leq t $$

D'abord, divisons la fonction objectif et la contrainte par $t$ et substituons le vecteur des coefficients comme suit $\mathbf{\beta} := \frac{1}{t}\mathbf{\beta}$. Cette étape nous permet d'absorber la constante $t$ entièrement dans la fonction objectif et de réécrire le problème de la façon suivante :

$$ \min_{\mathbf{\beta}} ||\mathbf{X \beta} − \frac{1}{t}\mathbf{y}||_2^2 + \lambda_2 ||\mathbf{\beta}||_2^2 \text{ tel que } |\mathbf{\beta}|_1 \leq 1 $$


Pour simplifier la contrainte L1, nous séparons $\mathbf{\beta}$ en deux ensemble de variables positives, les composantes positives $\mathbf{\beta^+} \geq 0$ et les composantes négatives $\mathbf{\beta^-} \geq 0$, de sorte que nous ayons $ \mathbf{\beta} = \mathbf{\beta^+} - \mathbf{\beta^-}$.

Ensuite, nous "empilons" $\mathbf{\beta^+}$ et $\mathbf{\beta^-}$ l'un sur l'autre et formons un nouveau vecteur de coefficients $\mathbf{\hat{\beta}} = [\mathbf{\beta^+}; \mathbf{\beta^-}] \in \mathbb{R}^{2p}_{\geq 0}$. Alors nous avons : 

$$ \min_{\mathbf{\hat{\beta}}_i \geq 0} ||[\mathbf{X}, -\mathbf{X}]\mathbf{\hat{\beta}}  − \frac{1}{t}\mathbf{y}||_2^2 + \lambda_2 \sum_{i=1}^{2p} \hat{\beta}_i^2 \text{ tel que } \sum_{i=1}^{2p} \hat{\beta}_i \leq 1$$

Ici l'ensemble $\mathbb{R}^{2p}_{\geq 0}$ désigne l'ensemble des vecteurs de $\mathbb{R}^{2p}$ de valeurs positives. Tant que $\lambda_2 \neq 0$, la solution au roblème est unique est satisfait, pour tout $i$, $\beta_i^+ = 0$ or $\beta_i^- = 0$.

Aussi, si $t$ devient très grand, le problème est équivalent à une régression RIDGE, qui a tendance à donner des solutions dense. Nous choisissons alors d'éviter ce cas où $t >> 0$. Ainsi, la contrainte L1 sera toujours resserée, ce qui signifie que $|\mathbf{\hat{\beta}}|_1 = 0$. Nous incorporons cette égalité dans le problème et nous obtenons :

$$ \min_{\mathbf{\hat{\beta}}_i \geq 0} ||[\mathbf{X}, -\mathbf{X}]\mathbf{\hat{\beta}}  − \frac{1}{t}\mathbf{y}||_2^2 + \lambda_2 \sum_{i=1}^{2p} \hat{\beta}_i^2 \text{ tel que } \sum_{i=1}^{2p} \hat{\beta}_i = 1$$

Nous introduisons maintenant les matrices $\mathbf{\hat{X}}_1 = \mathbf{X} - \frac{1}{t}\mathbf{y1}^T$ et $\mathbf{\hat{X}}_2 = \mathbf{X} + \frac{1}{t}\mathbf{y1}^T$, avec $\mathbf{1}$ le vecteur colonne de taille $p$ rempli de 1. 

Alors nous construisons la matrice $\mathbf{\hat{Z}} = [\mathbf{\hat{X}}_1, -\mathbf{\hat{X}}_2]$, tel que nous ayons $\mathbf{\hat{Z}\hat{\beta}} = [\mathbf{X}, -\mathbf{X}]\mathbf{\hat{\beta}}  − \frac{1}{t}\mathbf{y}$. Nous incorporons cela dans le problème et nous avons : 

$$ \min_{\mathbf{\hat{\beta}}_i \geq 0} ||\mathbf{\hat{Z}\hat{\beta}}||_2^2 + \lambda_2 \sum_{i=1}^{2p} \hat{\beta}_i^2 \text{ tel que } \sum_{i=1}^{2p} \hat{\beta}_i = 1$$

Dans la section "[4. SVMs](#svm)", nous montrons que nous pouvons obtenir la solution optimale $\mathbf{\hat{\beta}^*}$ à ce problème en construisant un jeu de données de classification binaire $\mathbf{\hat{X}}$ et $\mathbf{\hat{y}}$ tel que $\mathbf{\hat{\beta}^*} =  \frac{\mathbf{\alpha^*}}{|\mathbf{\alpha^*}|_1}$, avec $\mathbf{\alpha^*}$ la solution au problème SVM dual $\mathbf{\hat{X}}$ et $\mathbf{\hat{y}}$.

### B. Construction du jeu de données

Nous construisons un jeu de données de classification binaire avec $m = 2p$ exemples et $q = n$ variables correspondant aux colonnes de $\mathbf{\hat{X}} = [\mathbf{\hat{X}}_1, \mathbf{\hat{X}}_2]$. 

Remarquons que cela nous donne les données suivantes : $\{(\mathbf{\hat{x}}^{(1)},\hat{y}_1),\ldots,(\mathbf{\hat{x}}^{(2p)},\hat{y}_{2p})\}$, avec, pour tout $i$, $\mathbf{\hat{x}}^{(i)} \in \mathbb{R}^n$ et $\hat{y}_1, \ldots, \hat{y}_p = + 1$ et $\hat{y}_{p+1},...,\hat{y}_{2p} = −1$. En d'autres termes, les colonnes de $\mathbf{\hat{X}}_1$ sont de la classe $+1$ et les colonnes de $\mathbf{\hat{X}}_2$ de la classe $−1$. 

Nous voyons directement que pour $\mathbf{\hat{Z}} = [\mathbf{\hat{X}}_1, -\mathbf{\hat{X}}_2]$, nous avons $\mathbf{\hat{Z}} = (\hat{y}_1\mathbf{\hat{x}}_1, \ldots, \hat{y}_m\mathbf{\hat{x}}_m)$. 

### C. Solution optimale 

Notons $\mathbf{\alpha}^*$ la solution optimale au problème SVM dual appliqué à cette matrice $\mathbf{\hat{Z}}$ et avec $C = \frac{1}{2\lambda_2}$. Nous allons maintenant réduire le problème SVM dual à la dernière formulation du problème Elastic Net, sans changer la solution optimale $\mathbf{\alpha}^*$. D'abord, en connaissant la solution optimale au problème SVM dual, nous pouvons ajouter la constante $\sum_{i=1}^{2p} \alpha_i = |\mathbf{\alpha}^*|_1$, et le problème devient :

$$ \min_{\alpha_i \geq 0} ||\mathbf{Z\alpha}||^2_2 + \lambda_2 \sum_{i=1}^{2p} \alpha_i^2 - 2 \sum_{i=1}^{2p} \alpha_i \text{ tel que } \sum_{i=1}^{2p} \alpha_i = |\mathbf{\alpha}^*|_1$$

À cause de la contrainte d'égalité, nous pouvons laisser tomber le terme constant $-2 \sum_{i=1}^{2p} \alpha_i$. Enlever ce terme constant n'affecte pas la solution et amène au problème d'optimisation suivante :

$$ \min_{\alpha_i \geq 0} ||\mathbf{Z\alpha}||^2_2 + \lambda_2 \sum_{i=1}^{2p} \alpha_i^2 \text{ tel que } \sum_{i=1}^{2p} \alpha_i = |\mathbf{\alpha}^*|_1$$

Notons que si nous divisons la fonction objectif par $|\mathbf{\alpha}^*|_1^2$ et la constrainte par $|\mathbf{\alpha}^*|_1$, puis que nous introduisons un changement de variable $\beta_i = \frac{\alpha_i}{|\mathbf{\alpha}^*|_1}$, nous obtenons exactement la dernière formulation du problème Elastic Net, dont la solution optimale est : $\mathbf{\beta}^* = \frac{\mathbf{\alpha}^*}{|\mathbf{\alpha}^*|_1}$.

## 6. <a id="sven"></a> Implémentation de SVEN

L'algorithme correspondant à toutes les étapes que nous avons décrites et démontrées à la section précédente s'appelle le SVEN (pour Support Vector Elastic Net). En voici une implémentation en pseudocode : 

<div class="alert alert-success"> 

**SVEN Algorithm**
<br>
$\ \ \ \ $ | $\ \ \ \ $ **function** $\beta =\text{SVEN}(X, y, t, \lambda_2)$; 
<br>
$\ \ \ \ $ | $\ \ \ \ $ $n, p = \text{shape}(X)$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ $X_{\text{new}} = [X - (y ./ t) * \text{ones}(1, p); X +  (y ./ t) * \text{ones}(1, p)]^T$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ $y_{\text{new}} = [\text{ones}(p, 1); -\text{ones}(p, 1)]$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ **if** $2p > n$ **then**
<br>
$\ \ \ \ $ | $\ \ \ \ $ $\ \ \ \ $ $\ \ \ \ $ | $\ \ \ \ $ $w = \text{SVMPrimal}(X_{\text{new}}, y_{\text{new}}, C = 1/(2λ2))$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ $\ \ \ \ $ $\ \ \ \ $ | $\ \ \ \ $ $\alpha = C .* \max(\text{zeros}(2p, 1), \text{ones}(2p, 1)-y_{\text{new}}.*(X_{\text{new}} * w))$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ **else**
<br>
$\ \ \ \ $ | $\ \ \ \ $ $\ \ \ \ $ $\ \ \ \ $ | $\ \ \ \ $ $\alpha = \text{SVMDual}(X_{\text{new}}, y_{\text{new}}, C = 1/(2λ2))$;
<br>
$\ \ \ \ $ | $\ \ \ \ $ **end if**
<br>
$\ \ \ \ $ | $\ \ \ \ $ $\beta = t * (\alpha[1:p] - \alpha[p+1:2p])/\text{sum}(\alpha)$;
<br>
$\ \  $ **end**
</div>


<div class="alert alert-info">
    
#### Remarque
    
Nous pouvons remarquer que les formulations primale et duale du problème SVM ont différentes complexités temporelles. SVEN choisi donc la plus rapide en fonction de si $2p > n$ et vice-versa.

</div> 

<div class="alert alert-warning"> 
    
### Exercice :

**Implémentez une fonction `SVEN` qui renvoie le vecteur de coefficients $\beta$.** Vous pourrez vous aider des fonctions `SVMPrimal` et `SVMDual` déjà implémentées.**

</div>

In [ ]:
# %load solutions/code5.py
### Ecrivez votre code ici
# Si vous etes bloques, decommentez la premiere ligne et executez ce bloc.

#def SVEN(X, y, t, l2):

Testons notre fonction

In [ ]:
# We process the SVEN algorithm
t0 = time()
sven_beta = SVEN(X_train, y_train, t=2.0, l2=0.5)
print("> Time spent : {}s".format(time()-t0))

y_train_pred = np.dot(X_train, en_beta)
y_test_pred = np.dot(X_test, en_beta)
print("> Training set MSE = {}".format(mean_squared_error(y_train_pred, y_train)))
print("> Testing set MSE = {}\n".format(mean_squared_error(y_test_pred, y_test)))

<div class="alert alert-info">
    
On remarque que, pour les mêmes paramètres $t$ et $\lambda_2$, `SVEN` est plus rapide que `ElasticNet`, alors que les deux donnent exactement les mêmes solutions. 

</div>

In [ ]:
# visualization
train_colors = ['r' if l > 0 else 'b' for l in y_train_pred]

test_colors = ['r' if l > 0 else 'b' for l in y_test_pred]

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.scatter(X_train[:, 1], X_train[:, 3], color=train_colors)
ax1.set_title("Training set")

ax2.scatter(X_test[:, 1], X_test[:, 3], color=test_colors)
ax2.set_title("Training set")

plt.show()

## 7. <a id="more"></a> Optimisation

Les implémentations que nous avons faites ici de ces algorithmes sont loins d'être optimisées. En effet, on trouve aujourd'hui de nombreux programmes et/ou librairies qui proposent des versions optimisées de ces algorithmes. 

- [shotgun](https://github.com/akyrola/shotgun) : Package C++ proposant une implémentation parallélisé de descentes de gradient coordonnées pour la résolution de **LASSO**.

- [glmnet](https://web.stanford.edu/~hastie/glmnet_python/) : Package développé par Friedman qui propose une implémentation "single-core" (non parallélisée) du problème **Elastic Net**. Cette implémentation est adopte une stratégie de descentes de gradient coordonnées, qui est très difficile à paralléliser dans le cadre d'un problème Elastic Net.

- [libsvm](https://www.csie.ntu.edu.tw/~cjlin/libsvm/): un logiciel intégré qui résout des problèmes de classification et de régression par **SVM**. Il utilise un algorithme SMO (Sequential Minimal Optimization), qui n'est pas optimal dans le cas linéaire puisqu'il n'est pas parallélisé.

- [liblinear](https://www.csie.ntu.edu.tw/~cjlin/liblinear/): un logiciel intégré (analogue à libsvm) qui résout des problèmes de classification et de régression linéaires par **SVM**. Celui-ci propose une version distribuée. 

L'implémentation de l'algorithme `SVEN` avec des versions parallélisées des méthodes SVMs (comme dans liblinear par exemple) donne, dans un environnement de calcul distribué (GPUs, multicore-CPUs,...), de bien meilleurs résultats en terme de temps que les algorithmes déjà optimisé que sont shotgun et glmnet, pour des paramètres équivalents (et par conséquent des résultats de classification ou de régression équivalents aussi).

<a id="sources"></a> *Sources :*

**[A Reduction of the Elastic Net to Support Vector Machines with an Application to GPU Computing](https://arxiv.org/pdf/1409.1976)**<br>
Quan Zhou, Wenlin Chen, Shiji Song, Jacob R. Gardner, Kilian Q. Weinberger, Yixin Chen, 6 Sep 2014.
<br>**[Regularization and variable selection via the Elastic Net](https://web.stanford.edu/~hastie/TALKS/enet_talk.pdf)**
<br> Hui Zou and Trevor Hastie, Department of Statistics, Stanford University
<br>**[SVMs, bias-variance, and kernels](https://github.com/erachelson/MLclass/tree/master/3%20-%20SVMs%2C%20bias-variance%2C%20and%20kernels)**
<br> Notebook by Emmanuel Rachelson, 2018. License CC-BY-SA-NC.
<br>**[Scikit-Learn](https://scikit-learn.org/stable/)**
<br> © 2007 - 2018, scikit-learn developers (BSD License)
<br>**[Lasso (statistics)](https://en.wikipedia.org/wiki/Lasso_(statistics))**
<br> Wikipedia Article
<br>**[Elastic Net Regularization](https://en.wikipedia.org/wiki/Elastic_net_regularization)**
<br> Wikipedia Article
<br>**[Support-Vector Machine](https://en.wikipedia.org/wiki/Support-vector_machine)**
<br> Wikipedia Article
<br>**[UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/)**
<br> Open-source dataset repository, University of California, Irvine